In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

from tensorflow.keras.layers import Input, Activation, Dense, Flatten, RepeatVector, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import time
from datetime import datetime

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(88, 88),
        batch_size=40,
        color_mode="grayscale",
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'dataset/valid',
        target_size=(88, 88),    
        batch_size=40,
        color_mode="grayscale",
        class_mode='categorical')

Found 21000 images belonging to 7 classes.
Found 8240 images belonging to 7 classes.


In [11]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), input_shape=(88,88,1), kernel_initializer='he_normal'))
model.add(Conv2D(filters = 8, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.7))

model.add(Flatten())

model.add(Dense(128, kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.7))

model.add(Dense(7, activation='softmax', kernel_initializer='he_normal'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 86, 86, 16)        160       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 86, 86, 8)         136       
_________________________________________________________________
batch_normalization_4 (Batch (None, 86, 86, 8)         32        
_________________________________________________________________
activation_4 (Activation)    (None, 86, 86, 8)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 43, 43, 8)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 43, 43, 8)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 14792)            

In [13]:
early_stopping = EarlyStopping(monitor='val_loss',mode='min', patience=10)
logdir="logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=logdir)



In [14]:
model.fit_generator(
        train_generator,
        steps_per_epoch=525,
        epochs=15,
        validation_data=test_generator,
        validation_steps=206,
        callbacks=[early_stopping, tensorboard])

Epoch 1/15
525/525 [==============================] - 14s 27ms/step - accuracy: 0.8818 - loss: 0.3156 - val_loss: 0.2486 - val_accuracy: 0.9187
Epoch 2/15
525/525 [==============================] - 14s 26ms/step - accuracy: 0.8854 - loss: 0.3020 - val_loss: 2.4951 - val_accuracy: 0.5061
Epoch 3/15
525/525 [==============================] - 14s 26ms/step - accuracy: 0.8902 - loss: 0.2857 - val_loss: 0.2832 - val_accuracy: 0.9104
Epoch 4/15
525/525 [==============================] - 13s 24ms/step - accuracy: 0.9042 - loss: 0.2571 - val_loss: 0.1956 - val_accuracy: 0.9500
Epoch 5/15
525/525 [==============================] - 13s 24ms/step - accuracy: 0.9020 - loss: 0.2579 - val_loss: 0.8975 - val_accuracy: 0.7012
Epoch 6/15
525/525 [==============================] - 13s 24ms/step - accuracy: 0.9053 - loss: 0.2514 - val_loss: 0.9422 - val_accuracy: 0.6627
Epoch 7/15
525/525 [==============================] - 14s 26ms/step - accuracy: 0.9139 - loss: 0.2356 - val_loss: 0.4755 - val_accuracy:

In [23]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
accuracy: 94.50%


In [16]:
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
Instructions for updating:
Please use Model.predict, which supports generators.
{'1x1_brick': 0, '2x6_brick': 2, '1x4_brick': 1}
[[0.121 0.005 0.873]
 [0.001 0.001 0.998]
 [0.149 0.007 0.845]
 [0.001 0.009 0.991]
 [0.000 0.990 0.010]
 [0.001 0.959 0.041]
 [0.993 0.000 0.007]
 [0.005 0.795 0.200]
 [0.002 0.011 0.988]
 [0.027 0.073 0.900]
 [0.023 0.660 0.317]
 [0.021 0.680 0.299]
 [0.038 0.023 0.939]
 [0.994 0.000 0.006]
 [0.998 0.000 0.002]
 [0.986 0.000 0.014]
 [0.010 0.763 0.227]
 [0.998 0.000 0.002]
 [0.998 0.000 0.002]
 [0.998 0.000 0.002]
 [0.027 0.024 0.949]
 [0.001 0.001 0.998]
 [0.996 0.000 0.004]
 [0.017 0.734 0.249]
 [0.004 0.749 0.247]
 [0.998 0.000 0.002]
 [0.998 0.000 0.002]
 [0.995 0.000 0.005]
 [0.004 0.754 0.242]
 [0.000 0.964 0.035]
 [0.998 0.000 0.002]
 [0.001 0.957 0.043]
 [0.997 0.000 0.003]
 [0.008 0.836 0.156]
 [0.000 0.990 0.010]
 [0.985 0.000 0.015]
 [0.001 0.955 0.044]
 [0.995 0.000 0.005]
 [0.999 0.000 0.001]
 [0.989 0.000 0.011]
 [0.994 0.000 0.0

In [24]:
model.save("lego_sorter_v3.h5")